In [1]:
import pandas as pd
import numpy as np
import requests
import io
from datetime import datetime

## Pruebas de carga local

## Pruebas de carga desde una URL

## Inicio

In [2]:
url_items = "https://raw.githubusercontent.com/sidooms/MovieTweetings/master/latest/movies.dat"
url_ratin = "https://raw.githubusercontent.com/sidooms/MovieTweetings/master/latest/ratings.dat"
url_users = "https://raw.githubusercontent.com/sidooms/MovieTweetings/master/latest/users.dat"

itm_file = pd.read_csv(url_items, sep='::',encoding='utf8',error_bad_lines=False,engine='python')
use_file = pd.read_csv(url_users, sep='::',encoding='utf8',error_bad_lines=False,engine='python')
rat_file = pd.read_csv(url_ratin, sep='::',encoding='utf8',error_bad_lines=False,engine='python')

itm_file.columns = ['movie_id', 'movie_title', 'genre']
use_file.columns = ['userid', 'twitter_id']
rat_file.columns = ['user_id','movie_id','rating','rating_timestamp']

In [3]:
itm_file.head() 

,movie_id,movie_title,genre
0,10,La sortie des usines Lumière (1895),Documentary|Short
1,12,The Arrival of a Train (1896),Documentary|Short
2,25,The Oxford and Cambridge University Boat Race ...,NaN
3,91,Le manoir du diable (1896),Short|Horror
4,131,Une nuit terrible (1896),Short|Comedy|Horror


In [4]:
itm_file.shape

(34510, 3)

In [5]:
use_file.head()

,userid,twitter_id
0,2,40501255
1,3,417333257
2,4,138805259
3,5,2452094989
4,6,391774225


In [6]:
use_file.shape

(60452, 2)

In [7]:
use_file = use_file.rename(columns = {'userid':'user_id'})

In [8]:
rat_file.tail()

,user_id,movie_id,rating,rating_timestamp
816678,60451,8695030,1,1567816456
816679,60451,9398640,1,1569026144
816680,60452,816711,8,1371972851
816681,60453,1559547,2,1373287369
816682,60453,2415464,2,1373772560


In [9]:
rat_file.shape

(816683, 4)

In [10]:
rat_file['rating_timestamp'] = pd.to_datetime(rat_file['rating_timestamp'], unit='s')
rat_file.tail()

,user_id,movie_id,rating,rating_timestamp
816678,60451,8695030,1,2019-09-07 00:34:16
816679,60451,9398640,1,2019-09-21 00:35:44
816680,60452,816711,8,2013-06-23 07:34:11
816681,60453,1559547,2,2013-07-08 12:42:49
816682,60453,2415464,2,2013-07-14 03:29:20


In [11]:
rat_file = rat_file.rename(columns = {'rating_timestamp':'date'})
rat_file['time'] = rat_file['date'].astype(str)
rat_file['date'] = rat_file['date'].astype(str)

In [12]:
rat_file.time.dtype

dtype('O')

In [13]:
rat_file['date'] = rat_file['date'].str.split(' ').str[0]
rat_file['time'] = rat_file['time'].str[-8:]

In [14]:
rat_file.head()

,user_id,movie_id,rating,date,time
0,1,117060,7,2013-07-10,00:13:51
1,1,120755,6,2013-07-10,02:46:00
2,1,317919,6,2013-07-10,22:36:03
3,1,454876,10,2013-07-12,09:25:25
4,1,790724,8,2013-07-24,04:48:40


In [15]:
itm_file['movie_year'] = itm_file['movie_title']

In [16]:
itm_file['movie_title'] = itm_file['movie_title'].str.split('(').str[0]
itm_file['movie_title'] = itm_file['movie_title'].str.strip()
itm_file['movie_year'] = itm_file['movie_year'].str[-6:]

In [17]:
itm_file['movie_year'] = itm_file['movie_year'].str.split('(').str[1]
itm_file['movie_year'] = itm_file['movie_year'].str.split(')').str[0]

In [18]:
itm_file.head() 

,movie_id,movie_title,genre,movie_year
0,10,La sortie des usines Lumière,Documentary|Short,1895
1,12,The Arrival of a Train,Documentary|Short,1896
2,25,The Oxford and Cambridge University Boat Race,NaN,1895
3,91,Le manoir du diable,Short|Horror,1896
4,131,Une nuit terrible,Short|Comedy|Horror,1896


In [19]:
itm_file.shape

(34510, 4)

In [20]:
df = rat_file

In [21]:
df.head()

,user_id,movie_id,rating,date,time
0,1,117060,7,2013-07-10,00:13:51
1,1,120755,6,2013-07-10,02:46:00
2,1,317919,6,2013-07-10,22:36:03
3,1,454876,10,2013-07-12,09:25:25
4,1,790724,8,2013-07-24,04:48:40


In [22]:
df['date']=df['date'].astype(str)

In [23]:
df.head()

,user_id,movie_id,rating,date,time
0,1,117060,7,2013-07-10,00:13:51
1,1,120755,6,2013-07-10,02:46:00
2,1,317919,6,2013-07-10,22:36:03
3,1,454876,10,2013-07-12,09:25:25
4,1,790724,8,2013-07-24,04:48:40


In [24]:
df['date']= pd.to_datetime(df['date']) 

In [25]:
df['date'].unique()

array(['2013-07-10T00:00:00.000000000', '2013-07-12T00:00:00.000000000',
       '2013-07-24T00:00:00.000000000', ...,
       '2014-12-11T00:00:00.000000000', '2015-09-25T00:00:00.000000000',
       '2015-09-30T00:00:00.000000000'], dtype='datetime64[ns]')

In [26]:
df.head()

,user_id,movie_id,rating,date,time
0,1,117060,7,2013-07-10,00:13:51
1,1,120755,6,2013-07-10,02:46:00
2,1,317919,6,2013-07-10,22:36:03
3,1,454876,10,2013-07-12,09:25:25
4,1,790724,8,2013-07-24,04:48:40


In [27]:
df = df.sort_values(by='date')
df.reset_index(drop=True, inplace=True)
df.head()

,user_id,movie_id,rating,date,time
0,15737,95631,10,2013-02-28,23:20:01
1,28364,265349,8,2013-02-28,23:55:41
2,51099,1853728,9,2013-02-28,18:41:35
3,18522,313542,7,2013-02-28,17:59:49
4,34662,97733,7,2013-02-28,22:06:37


In [28]:
df_2 = pd.merge(df,itm_file, on='movie_id')

In [29]:
df_2.head()

,user_id,movie_id,rating,date,time,movie_title,genre,movie_year
0,15737,95631,10,2013-02-28,23:20:01,Midnight Run,Action|Comedy|Crime|Thriller,1988
1,14737,95631,7,2013-03-12,03:50:12,Midnight Run,Action|Comedy|Crime|Thriller,1988
2,14577,95631,7,2013-05-16,19:53:56,Midnight Run,Action|Comedy|Crime|Thriller,1988
3,21712,95631,8,2013-06-17,22:15:38,Midnight Run,Action|Comedy|Crime|Thriller,1988
4,8074,95631,7,2013-09-26,13:51:48,Midnight Run,Action|Comedy|Crime|Thriller,1988


In [30]:
df_final =pd.merge(df_2, use_file, on='user_id')
df_final = df_final.sort_values(by='date')
df_final.reset_index(drop=True, inplace=True)
df_final

,user_id,movie_id,rating,date,time,movie_title,genre,movie_year,twitter_id
0,15737,95631,10,2013-02-28,23:20:01,Midnight Run,Action|Comedy|Crime|Thriller,1988,19039635
1,2232,1853728,9,2013-02-28,23:49:26,Django Unchained,Drama|Western,2012,121246530
2,16730,1981677,5,2013-02-28,21:47:13,Pitch Perfect,Comedy|Music|Romance,2012,230985233
3,3726,93389,9,2013-02-28,23:19:58,The Last Emperor,Biography|Drama|History,1987,600317970
4,3674,1277953,7,2013-02-28,21:32:09,Madagascar 3: Europe's Most Wanted,Animation|Adventure|Comedy|Family,2012,66723750
5,32916,975645,7,2013-02-28,21:09:50,Hitchcock,Biography|Comedy|Drama|Romance,2012,317396727
6,53692,67989,8,2013-02-28,16:24:58,Wild Rovers,Western,1971,110872248
7,17799,1454029,8,2013-02-28,21:12:36,The Help,Drama,2011,397449062
8,41759,2171847,6,2013-02-28,14:38:27,Dead Mine,Action|Horror,2012,105210134
9,45875,884328,7,2013-02-28,15:08:24,The Mist,Horror|Sci-Fi,2007,206144706


In [31]:
df_cross = pd.DataFrame(df_final.genre)

numeration = int(len(df_final))+1
df_cross['id'] = pd.Series(range(1,numeration))

In [32]:
df_cross

,genre,id
0,Action|Comedy|Crime|Thriller,1
1,Drama|Western,2
2,Comedy|Music|Romance,3
3,Biography|Drama|History,4
4,Animation|Adventure|Comedy|Family,5
5,Biography|Comedy|Drama|Romance,6
6,Western,7
7,Drama,8
8,Action|Horror,9
9,Horror|Sci-Fi,10


In [33]:
df_cross.isnull().values.any()

True

In [34]:
df_cross.isnull().sum()

genre    442
id         0
dtype: int64

In [35]:
df_cross['genre'] = df_cross.fillna('noGenre')

In [36]:
newdf_cross = pd.concat([pd.Series(row['id'], row['genre'].split('|'))
           for _, row in df_cross.iterrows()]).reset_index()

In [37]:
newdf_cross.columns

Index(['index', 0], dtype='object')

In [38]:
new_dfcross = pd.crosstab(newdf_cross[0], newdf_cross['index'])

In [39]:
new_dfcross

index,Action,Adult,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,...,Reality-TV,Romance,Sci-Fi,Short,Sport,Talk-Show,Thriller,War,Western,noGenre
0,,,,,,,,,,,,,,,,,,,,,
1,1,0,0,0,0,1,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,0,0,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,1,1,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,1,1,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
8,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
9,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [40]:
new_dfcross.rename(columns={'index': 'genre', 0: 'id'}, inplace=True)

In [41]:
new_dfcross.columns

Index(['Action', 'Adult', 'Adventure', 'Animation', 'Biography', 'Comedy',
       'Crime', 'Documentary', 'Drama', 'Family', 'Fantasy', 'Film-Noir',
       'Game-Show', 'History', 'Horror', 'Music', 'Musical', 'Mystery', 'News',
       'Reality-TV', 'Romance', 'Sci-Fi', 'Short', 'Sport', 'Talk-Show',
       'Thriller', 'War', 'Western', 'noGenre'],
      dtype='object', name='index')

In [42]:
result = pd.merge(df_final.reset_index(),
                  new_dfcross.reset_index(), 
                  left_index=True, 
                  right_index=True)
result

,index,user_id,movie_id,rating,date,time,movie_title,genre,movie_year,twitter_id,...,Reality-TV,Romance,Sci-Fi,Short,Sport,Talk-Show,Thriller,War,Western,noGenre
0,0,15737,95631,10,2013-02-28,23:20:01,Midnight Run,Action|Comedy|Crime|Thriller,1988,19039635,...,0,0,0,0,0,0,1,0,0,0
1,1,2232,1853728,9,2013-02-28,23:49:26,Django Unchained,Drama|Western,2012,121246530,...,0,0,0,0,0,0,0,0,1,0
2,2,16730,1981677,5,2013-02-28,21:47:13,Pitch Perfect,Comedy|Music|Romance,2012,230985233,...,0,1,0,0,0,0,0,0,0,0
3,3,3726,93389,9,2013-02-28,23:19:58,The Last Emperor,Biography|Drama|History,1987,600317970,...,0,0,0,0,0,0,0,0,0,0
4,4,3674,1277953,7,2013-02-28,21:32:09,Madagascar 3: Europe's Most Wanted,Animation|Adventure|Comedy|Family,2012,66723750,...,0,0,0,0,0,0,0,0,0,0
5,5,32916,975645,7,2013-02-28,21:09:50,Hitchcock,Biography|Comedy|Drama|Romance,2012,317396727,...,0,1,0,0,0,0,0,0,0,0
6,6,53692,67989,8,2013-02-28,16:24:58,Wild Rovers,Western,1971,110872248,...,0,0,0,0,0,0,0,0,1,0
7,7,17799,1454029,8,2013-02-28,21:12:36,The Help,Drama,2011,397449062,...,0,0,0,0,0,0,0,0,0,0
8,8,41759,2171847,6,2013-02-28,14:38:27,Dead Mine,Action|Horror,2012,105210134,...,0,0,0,0,0,0,0,0,0,0
9,9,45875,884328,7,2013-02-28,15:08:24,The Mist,Horror|Sci-Fi,2007,206144706,...,0,0,1,0,0,0,0,0,0,0


In [43]:
result.columns

Index([      'index',     'user_id',    'movie_id',      'rating',
              'date',        'time', 'movie_title',       'genre',
        'movie_year',  'twitter_id',             0,      'Action',
             'Adult',   'Adventure',   'Animation',   'Biography',
            'Comedy',       'Crime', 'Documentary',       'Drama',
            'Family',     'Fantasy',   'Film-Noir',   'Game-Show',
           'History',      'Horror',       'Music',     'Musical',
           'Mystery',        'News',  'Reality-TV',     'Romance',
            'Sci-Fi',       'Short',       'Sport',   'Talk-Show',
          'Thriller',         'War',     'Western',     'noGenre'],
      dtype='object')

In [44]:
itm_file.movie_id.unique().shape

(34508,)

In [45]:
itm_file.duplicated('movie_id')

0        False
1        False
2        False
3        False
4        False
5        False
6        False
7        False
8        False
9        False
10       False
11       False
12       False
13       False
14       False
15       False
16       False
17       False
18       False
19       False
20       False
21       False
22       False
23       False
24       False
25       False
26       False
27       False
28       False
29       False
         ...  
34480    False
34481    False
34482    False
34483    False
34484    False
34485    False
34486    False
34487    False
34488    False
34489    False
34490    False
34491    False
34492    False
34493    False
34494    False
34495    False
34496    False
34497    False
34498    False
34499    False
34500    False
34501    False
34502    False
34503    False
34504    False
34505    False
34506    False
34507    False
34508    False
34509    False
Length: 34510, dtype: bool

In [46]:
result.movie_id.unique().shape

(34508,)

In [118]:
mv_df = itm_file

In [119]:
tes_coid = pd.DataFrame(data = pd.value_counts(result[['movie_id']].values.ravel()),columns=['count'])
tes_coid.shape

(34508, 1)

In [120]:
tes_coid.head()

,count
1454468,3067
816692,2743
993846,2737
770828,2663
816711,2402


In [121]:
tes_coid.reset_index(level=0, inplace=True) 
tes_coid.head()

,index,count
0,1454468,3067
1,816692,2743
2,993846,2737
3,770828,2663
4,816711,2402


In [122]:
tes_coid.columns = ['movie_id','count']

In [123]:
dolar = result['movie_title'] == '$'
result_dolar = result[dolar]
result_dolar

,index,user_id,movie_id,rating,date,time,movie_title,genre,movie_year,twitter_id,...,Reality-TV,Romance,Sci-Fi,Short,Sport,Talk-Show,Thriller,War,Western,noGenre
20481,20481,47429,68152,8,2013-04-07,22:16:20,$,Comedy|Crime|Drama,1971,14782875,...,0,0,0,0,0,0,0,0,0,0
230500,230500,4261,68152,7,2014-04-18,12:41:46,$,Comedy|Crime|Drama,1971,78128381,...,0,0,0,0,0,0,0,0,0,0
375312,375312,17101,68152,4,2015-04-25,03:05:01,$,Comedy|Crime|Drama,1971,532582706,...,0,0,0,0,0,0,0,0,0,0


In [124]:
grouped_data = result.groupby('movie_id')
rating = pd.DataFrame(grouped_data['rating'].mean().round(2))
rating.reset_index(level=0, inplace=True) 
rating.head()

,movie_id,rating
0,10,10.0
1,12,10.0
2,25,8.0
3,91,6.0
4,131,7.0


In [125]:
rating.shape

(34508, 2)

In [126]:
tes_coid = pd.merge(tes_coid,rating, on='movie_id',how='outer')
tes_coid = pd.merge(mv_df,tes_coid, on='movie_id',how='outer')
tes_coid

,movie_id,movie_title,genre,movie_year,count,rating
0,10,La sortie des usines Lumière,Documentary|Short,1895,1,10.00
1,12,The Arrival of a Train,Documentary|Short,1896,1,10.00
2,25,The Oxford and Cambridge University Boat Race,NaN,1895,1,8.00
3,91,Le manoir du diable,Short|Horror,1896,3,6.00
4,131,Une nuit terrible,Short|Comedy|Horror,1896,1,7.00
5,417,Le voyage dans la lune,Short|Action|Adventure|Comedy|Fantasy|Sci-Fi,1902,22,8.50
6,439,The Great Train Robbery,Short|Action|Crime|Western,1903,6,7.00
7,443,"Hiawatha, the Messiah of the Ojibway",NaN,1903,1,8.00
8,628,The Adventures of Dollie,Action|Short,1908,2,4.50
9,833,The Country Doctor,Short|Drama,1909,1,3.00


In [58]:
df = tes_coid[tes_coid['genre'] == 'Romance']
df.sort_values(by=['count'],ascending=False)

(29,)

In [59]:
vote_counts = tes_coid[tes_coid['count'].notnull()]['count'].astype('int')
vote_avergs = tes_coid[tes_coid['rating'].notnull()]['rating'].astype('int')
C = vote_avergs.mean()
C

6.460909881193857

In [60]:
m= vote_counts.quantile(0.95)
m

89.54999999999563

In [61]:
qualified = tes_coid[(tes_coid['count'] >= m) &
                  (tes_coid['count'].notnull())&
                  (tes_coid['rating'].notnull())][['movie_title',
                                                'movie_year',
                                                'count',
                                                'rating',
                                                'genre']]
qualified['count'] = qualified['count'].astype('int')
qualified['rating'] = qualified['rating'].astype('int')
qualified.shape

(1726, 5)

In [62]:
def weighted_rating(x):
    V = x['count']
    R = x['rating']
    return (V/(V+m) * R) +(m/(m+V) * C)

In [63]:
qualified['wr'] = qualified.apply(weighted_rating, axis=1)

In [64]:
qualified = qualified.sort_values('wr', ascending = False).head(250)

### Top Movies

In [65]:
qualified.head(15)

,movie_title,movie_year,count,rating,genre,wr
33189,Joker,2019,2232,9,Crime|Drama|Thriller,8.902059
28648,Avengers: Endgame,2019,1363,9,Action|Adventure|Fantasy|Sci-Fi,8.843465
8425,The Shawshank Redemption,1994,998,9,Drama,8.790929
17993,Inception,2010,875,9,Action|Adventure|Sci-Fi|Thriller,8.764268
8271,Forrest Gump,1994,774,9,Drama|Romance,8.736697
2037,12 Angry Men,1957,691,9,Crime|Drama,8.708698
14476,The Dark Knight,2008,664,9,Action|Crime|Drama|Thriller,8.698261
3891,The Godfather,1972,592,9,Crime|Drama,8.666385
9477,The Green Mile,1999,438,9,Crime|Drama|Fantasy|Mystery,8.568997
8118,Schindler's List,1993,414,9,Biography|Drama|History,8.548455


In [66]:
joker = itm_file['movie_title'] == 'Joker'
df_joker = itm_file[joker]
df_joker

,movie_id,movie_title,genre,movie_year
21113,1918886,Joker,Comedy|Family|Sci-Fi,2012
33189,7286456,Joker,Crime|Drama|Thriller,2019


In [67]:
itm_file['movie_id'].unique().shape

(34508,)

In [68]:
itm_file['movie_title'].unique().shape

(33116,)

In [69]:
s = tes_coid['genre'].str.split('|').apply(pd.Series, 1).stack()
s

0      0    Documentary
       1          Short
1      0    Documentary
       1          Short
3      0          Short
       1         Horror
4      0          Short
       1         Comedy
       2         Horror
5      0          Short
       1         Action
       2      Adventure
       3         Comedy
       4        Fantasy
       5         Sci-Fi
6      0          Short
       1         Action
       2          Crime
       3        Western
8      0         Action
       1          Short
9      0          Short
       1          Drama
10     0          Short
       1         Horror
       2         Sci-Fi
11     0          Short
       1          Drama
       2        Romance
12     0          Drama
               ...     
34490  1          Music
34491  0          Drama
34492  0         Comedy
34493  0    Documentary
       1          Drama
       2        Mystery
34494  0          Short
       1         Action
       2      Adventure
       3        Fantasy
       4        

In [131]:
s = tes_coid['genre'].str.split('|').apply(pd.Series, 1).stack()
s.index = s.index.droplevel(-1)
s.name = 'genres'
tes_coid.join(s.apply(lambda x: pd.Series(x.split('|'))))
tes_coid = tes_coid.join(s)
tes_coid

,movie_id,movie_title,genre,movie_year,count,rating,genres
0,10,La sortie des usines Lumière,Documentary|Short,1895,1,10.00,Documentary
0,10,La sortie des usines Lumière,Documentary|Short,1895,1,10.00,Short
1,12,The Arrival of a Train,Documentary|Short,1896,1,10.00,Documentary
1,12,The Arrival of a Train,Documentary|Short,1896,1,10.00,Short
2,25,The Oxford and Cambridge University Boat Race,NaN,1895,1,8.00,NaN
3,91,Le manoir du diable,Short|Horror,1896,3,6.00,Short
3,91,Le manoir du diable,Short|Horror,1896,3,6.00,Horror
4,131,Une nuit terrible,Short|Comedy|Horror,1896,1,7.00,Short
4,131,Une nuit terrible,Short|Comedy|Horror,1896,1,7.00,Comedy
4,131,Une nuit terrible,Short|Comedy|Horror,1896,1,7.00,Horror


In [ ]:
qualified = tes_coid[(tes_coid['count'] >= m) &
                  (tes_coid['count'].notnull())&
                  (tes_coid['rating'].notnull())][['movie_title',
                                                'movie_year',
                                                'count',
                                                'rating',
                                                'genre']]
qualified['count'] = qualified['count'].astype('int')
qualified['rating'] = qualified['rating'].astype('int')
qualified.shape 
def weighted_rating(x):
    V = x['count']
    R = x['rating']
    return (V/(V+m) * R) +(m/(m+V) * C)

In [132]:
def build_chart(genre,percentile = 0.85):
    df = tes_coid[tes_coid['genres'] == genre]
    vote_counts = df[df['count'].notnull()]['count'].astype('int')
    vote_averag = df[df['rating'].notnull()]['rating'].astype('int')
    C = vote_averag.mean()
    m = vote_counts.quantile(percentile)
    print(m)
    qualified = df[(df['count']>=m)&
                   (df['count'].notnull()) &
                   (df['rating'].notnull())][['movie_title',
                                              'movie_year',
                                              'count',
                                              'rating',
                                              'genre']]
    qualified['count'] = qualified['count'].astype('int')
    qualified['rating'] = qualified['rating'].astype('int')
    
    qualified['wr'] = qualified.apply(lambda x:(x['count']/(x['count']+m)*x['rating'])+(m/(m+x['count'])*C), axis = 1)
    qualified = qualified.sort_values('wr', ascending = False).head(250)
    

    return qualified

In [128]:
m

89.54999999999563

In [129]:
C

6.460909881193857

In [136]:
build_chart('Sci-Fi').head(15)

67.0


,movie_title,movie_year,count,rating,genre,wr
28648,Avengers: Endgame,2019,1363,9,Action|Adventure|Fantasy|Sci-Fi,8.853379
17993,Inception,2010,875,9,Action|Adventure|Sci-Fi|Thriller,8.777423
7642,Terminator 2: Judgment Day,1991,234,9,Action|Sci-Fi,8.303429
18410,Gravity,2013,3067,8,Drama|Sci-Fi|Thriller,7.954477
15338,Interstellar,2014,2743,8,Adventure|Drama|Sci-Fi,7.949228
18074,Mad Max: Fury Road,2015,1911,8,Action|Adventure|Sci-Fi|Thriller,7.927873
18151,Star Trek Into Darkness,2013,1879,8,Action|Adventure|Sci-Fi,7.926687
18281,Deadpool,2016,1843,8,Action|Adventure|Comedy|Sci-Fi,7.925305
20490,Her,2013,1771,8,Drama|Romance|Sci-Fi,7.922379
20967,X-Men: Days of Future Past,2014,1752,8,Action|Adventure|Sci-Fi|Thriller,7.921568


## Content based recommendation system

In [ ]:
s = pd.concat([pd.Series(row['id'], row['genre'].split('|'))
           for _, row in df_cross.iterrows()]).reset_index()

#  Pruebas

In [48]:
df_testing = pd.DataFrame(df_final.genre)

pruebas = df_testing.head()
jj = int(pruebas.count()+1)
pruebas['id'] = pd.Series(range(1,jj))

C:\Users\alter\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [49]:
pruebas

,genre,id
0,Action|Adventure|Mystery|Sci-Fi|Thriller,1
1,Drama|Fantasy|Horror|Mystery|Thriller,2
2,Action|Adventure|Fantasy|Horror|Sci-Fi|Thriller,3
3,Action|Adventure|Sci-Fi,4
4,Action|Adventure|Drama|Thriller,5


In [50]:
pd.concat([pd.Series(row['id'], row['genre'].split('|'))
           for _, row in pruebas.iterrows()]).reset_index()

,index,0
0,Action,1
1,Adventure,1
2,Mystery,1
3,Sci-Fi,1
4,Thriller,1
5,Drama,2
6,Fantasy,2
7,Horror,2
8,Mystery,2
9,Thriller,2


In [51]:
import pandas as pd
df = pd.DataFrame(['a b c']*100000, columns=['col'])
df.head()

,col
0,a b c
1,a b c
2,a b c
3,a b c
4,a b c


In [52]:
print (pd.DataFrame(dict(zip(range(3), [df['col'].apply(lambda x : x.split(' ')[i]) for i in range(3)]))).head())

   0  1  2
0  a  b  c
1  a  b  c
2  a  b  c
3  a  b  c
4  a  b  c


In [53]:
s = pruebas['genre'].str.split('|')
s

0       [Action, Adventure, Mystery, Sci-Fi, Thriller]
1          [Drama, Fantasy, Horror, Mystery, Thriller]
2    [Action, Adventure, Fantasy, Horror, Sci-Fi, T...
3                          [Action, Adventure, Sci-Fi]
4                 [Action, Adventure, Drama, Thriller]
Name: genre, dtype: object

In [54]:
q = pruebas['genre'].str.split('|').tolist()
q

[['Action', 'Adventure', 'Mystery', 'Sci-Fi', 'Thriller'],
 ['Drama', 'Fantasy', 'Horror', 'Mystery', 'Thriller'],
 ['Action', 'Adventure', 'Fantasy', 'Horror', 'Sci-Fi', 'Thriller'],
 ['Action', 'Adventure', 'Sci-Fi'],
 ['Action', 'Adventure', 'Drama', 'Thriller']]

In [55]:
pd.DataFrame(dict(zip(range(16), [pruebas['genre'].apply(lambda x : x.split('|')[i]) for i in range(2)])))

,0,1
0,Action,Adventure
1,Drama,Fantasy
2,Action,Adventure
3,Action,Adventure
4,Action,Adventure


In [56]:
s = pruebas['genre'].str.split('|').apply(pd.Series, 1).stack()
s

0  0       Action
   1    Adventure
   2      Mystery
   3       Sci-Fi
   4     Thriller
1  0        Drama
   1      Fantasy
   2       Horror
   3      Mystery
   4     Thriller
2  0       Action
   1    Adventure
   2      Fantasy
   3       Horror
   4       Sci-Fi
   5     Thriller
3  0       Action
   1    Adventure
   2       Sci-Fi
4  0       Action
   1    Adventure
   2        Drama
   3     Thriller
dtype: object

In [57]:
s.index = s.index.droplevel(-1)
s.name = 'genres'
s

0       Action
0    Adventure
0      Mystery
0       Sci-Fi
0     Thriller
1        Drama
1      Fantasy
1       Horror
1      Mystery
1     Thriller
2       Action
2    Adventure
2      Fantasy
2       Horror
2       Sci-Fi
2     Thriller
3       Action
3    Adventure
3       Sci-Fi
4       Action
4    Adventure
4        Drama
4     Thriller
Name: genres, dtype: object

In [58]:
# s = pruebas['genre']
s = pruebas['genre'].str.split('|').apply(pd.Series, 1).stack()
s.index = s.index.droplevel(-1)
s.name = 'genres'
pruebas.join(s.apply(lambda x: pd.Series(x.split('|'))))
pruebas = pruebas.join(s)
pruebas

,genre,id,genres
0,Action|Adventure|Mystery|Sci-Fi|Thriller,1,Action
0,Action|Adventure|Mystery|Sci-Fi|Thriller,1,Adventure
0,Action|Adventure|Mystery|Sci-Fi|Thriller,1,Mystery
0,Action|Adventure|Mystery|Sci-Fi|Thriller,1,Sci-Fi
0,Action|Adventure|Mystery|Sci-Fi|Thriller,1,Thriller
1,Drama|Fantasy|Horror|Mystery|Thriller,2,Drama
1,Drama|Fantasy|Horror|Mystery|Thriller,2,Fantasy
1,Drama|Fantasy|Horror|Mystery|Thriller,2,Horror
1,Drama|Fantasy|Horror|Mystery|Thriller,2,Mystery
1,Drama|Fantasy|Horror|Mystery|Thriller,2,Thriller


In [59]:
uni_gen = pruebas['genres'].unique()
uni_gen

array(['Action', 'Adventure', 'Mystery', 'Sci-Fi', 'Thriller', 'Drama',
       'Fantasy', 'Horror'], dtype=object)

In [60]:
process = pd.get_dummies(pd.Series(list(pruebas['genres'])))

In [61]:
result = pd.merge(pruebas.reset_index(),
                  process.reset_index(), 
                  left_index=True, 
                  right_index=True)
result

,index_x,genre,id,genres,index_y,Action,Adventure,Drama,Fantasy,Horror,Mystery,Sci-Fi,Thriller
0,0,Action|Adventure|Mystery|Sci-Fi|Thriller,1,Action,0,1,0,0,0,0,0,0,0
1,0,Action|Adventure|Mystery|Sci-Fi|Thriller,1,Adventure,1,0,1,0,0,0,0,0,0
2,0,Action|Adventure|Mystery|Sci-Fi|Thriller,1,Mystery,2,0,0,0,0,0,1,0,0
3,0,Action|Adventure|Mystery|Sci-Fi|Thriller,1,Sci-Fi,3,0,0,0,0,0,0,1,0
4,0,Action|Adventure|Mystery|Sci-Fi|Thriller,1,Thriller,4,0,0,0,0,0,0,0,1
5,1,Drama|Fantasy|Horror|Mystery|Thriller,2,Drama,5,0,0,1,0,0,0,0,0
6,1,Drama|Fantasy|Horror|Mystery|Thriller,2,Fantasy,6,0,0,0,1,0,0,0,0
7,1,Drama|Fantasy|Horror|Mystery|Thriller,2,Horror,7,0,0,0,0,1,0,0,0
8,1,Drama|Fantasy|Horror|Mystery|Thriller,2,Mystery,8,0,0,0,0,0,1,0,0
9,1,Drama|Fantasy|Horror|Mystery|Thriller,2,Thriller,9,0,0,0,0,0,0,0,1


In [62]:
new_df = pd.crosstab(result['id'], result['genres'])

In [63]:
new_df

genres,Action,Adventure,Drama,Fantasy,Horror,Mystery,Sci-Fi,Thriller
id,,,,,,,,
1,1,1,0,0,0,1,1,1
2,0,0,1,1,1,1,0,1
3,1,1,0,1,1,0,1,1
4,1,1,0,0,0,0,1,0
5,1,1,1,0,0,0,0,1


In [64]:
data = pd.DataFrame(columns=(uni_gen))
data['id'] = pd.Series(range(1,jj))
data

,Action,Adventure,Mystery,Sci-Fi,Thriller,Drama,Fantasy,Horror,id
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5


In [65]:
df_final.shape

(816306, 9)

In [66]:
df_final['movie_title'].nunique()
#34314

33093

In [67]:
lista_m = itm_file['movie_title']

lista_f = df_final['movie_title']

In [68]:
df_final[['dID','hID']].groupby(df['mID']).agg(['count', 'size', 'nunique'])

KeyError: "['dID' 'hID'] not in index"

In [ ]:
lista_m.iloc['Joker']

is_male = lista_m.loc[:,] == 'Joker'
df_male = lista_m.loc[is_male]
df_male.head()

In [ ]:
lista_f = pd.DataFrame(lista_f)
lista_f

In [ ]:
#surveys_df.loc[[0, 10], :] surveys_df[surveys_df.year == 2002]

lista_m = pd.DataFrame(lista_m)
lista_m['movie_title'].iloc[2]

In [ ]:
lista_m.movie_title.dtypes

In [ ]:
lista_m.movie_title =='The Oxford and Cambridge University Boat Race'

In [ ]:
is_joker = lista_m.loc[:, 'movie_title'] == 'Upstarts'
df_joker = lista_m.loc[is_joker]
df_joker.head()

In [ ]:
is_joker.unique()

In [ ]:
comparacion = []
for item in lista_m:
    if item in lista_m:
        comparacion.append(item)

if len(comparacion) > 0:
    print('Ambas listas contienen estos elementos')
    for item in comparacion: print('%s' % item)

else:
    print('No existe ningun elemento igual en las listas')

In [ ]:
lista1=["paco","pepe","luis"]

lista2=["diego","mari","luis"]

comparacion = []

 

for item in lista1:

  if item in lista2:

    comparacion.append(item)

 

if len(comparacion) > 0:

  print 'Ambas listas contienen estos elementos'

  for item in comparacion: print '%s' % item

else:

  print 'No existe ningun elemento igual en las listas'

In [ ]:
n_col = df_final.columns

for col in n_col:
    print(df[col].unique)